In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
import math
import matplotlib.pyplot as plt

In [2]:
str_files = ''

#df = pd.read_csv(str_files + "ph.csv",encoding='utf-8',sep=';')
dateparse = lambda dates:pd.datetime.strptime(dates,'%d/%m/%Y')
df = pd.read_csv(str_files + 'turbidez.CSV',encoding='utf-8',sep=';', parse_dates = ['data_coleta'],index_col=['data_coleta'],date_parser=dateparse)

In [3]:
z = df.loc[df['UGRHI'] == '10 - SOROCABA/MEDIO TIETE']

In [4]:
z = z.dropna(axis=1,how='any')

In [5]:
trainingd = z.resample('3M').mean()
trainingd
data = trainingd.iloc[:,1:2].values
train = trainingd.iloc[0:132,1:2].values  
test = trainingd.iloc[132:,1:2].values   
print('treino:',len(train))
print('teste:',len(test))

treino: 132
teste: 34


In [6]:
sc = MinMaxScaler()
train = sc.fit_transform(train)
test = sc.fit_transform(test)

In [7]:
def prepare_data(data, lags):
    X, y = [], []
    for row in range(len(data) - lags - 1):
        a = data[row:(row + lags), 0]
        X.append(a)
        y.append(data[row + lags, 0])
    return np.array(X), np.array(y)  

In [8]:
lags = 1
train_X, train_y = prepare_data(train, lags)
test_X, test_y = prepare_data(test, lags)

train_X = np.reshape(train_X, (train_X.shape[0],1,train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0],1,test_X.shape[1]))
print('Novo formato de treino:')
print(train_X.shape) 

Novo formato de treino:
(130, 1, 1)


In [9]:
model = Sequential() 

model.add(LSTM(units = 10, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 10, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 10, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='auto',verbose = 1,patience=5)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 20s 153ms/sample - loss: 0.0488 - val_loss: 0.0316
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0455 - val_loss: 0.0291
Epoch 3/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0426 - val_loss: 0.0268
Epoch 4/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0401 - val_loss: 0.0246
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0372 - val_loss: 0.0226
Epoch 6/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0348 - val_loss: 0.0209
Epoch 7/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0326 - val_loss: 0.0193
Epoch 8/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0309 - val_loss: 0.0181
Epoch 9/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.0291 - val_loss: 0.0172
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 2ms/sample - loss: 0.0237 - val_loss: 0.0158
Epoch 80/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0231 - val_loss: 0.0157
Epoch 81/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0240 - val_loss: 0.0157
Epoch 82/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0238 - val_loss: 0.0156
Epoch 83/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0233 - val_loss: 0.0157
Epoch 84/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0232 - val_loss: 0.0158
Epoch 85/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0231 - val_loss: 0.0160
Epoch 86/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0226 - val_loss: 0.0160
Epoch 87/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0234 - val_loss: 0.0161
Epoch 88/100
130/130 [==============================] - 0s 2ms/sample - loss:

In [10]:
train_score10 = model.evaluate(train_X, train_y, verbose=0)
print(train_score10)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score10, math.sqrt(train_score10)))
test_score10 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score10, math.sqrt(test_score10)))

0.022882028891203495
Pontuação de Treino: 0.02 MSE (0.15 RMSE)
Pontuação de teste: 0.02 MSE (0.13 RMSE)


In [11]:
model = Sequential() 

model.add(LSTM(units = 25, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 25, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 25, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='min',verbose = 1)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 19s 144ms/sample - loss: 0.0485 - val_loss: 0.0307
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0442 - val_loss: 0.0274
Epoch 3/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.0400 - val_loss: 0.0243
Epoch 4/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.0364 - val_loss: 0.0214
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0327 - val_loss: 0.0191
Epoch 6/100
130/130 [==============================] - 1s 4ms/sample - loss: 0.0299 - val_loss: 0.0173
Epoch 7/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.0277 - val_loss: 0.0161
Epoch 8/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0256 - val_loss: 0.0154
Epoch 9/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0239 - val_loss: 0.0154
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 2ms/sample - loss: 0.0230 - val_loss: 0.0160
Epoch 80/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0229 - val_loss: 0.0158
Epoch 81/100
130/130 [==============================] - 0s 3ms/sample - loss: 0.0229 - val_loss: 0.0158
Epoch 82/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0232 - val_loss: 0.0158
Epoch 83/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0233 - val_loss: 0.0158
Epoch 84/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0225 - val_loss: 0.0158
Epoch 85/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0226 - val_loss: 0.0160
Epoch 86/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0234 - val_loss: 0.0163
Epoch 87/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0231 - val_loss: 0.0163
Epoch 88/100
130/130 [==============================] - 0s 2ms/sample - loss:

In [12]:
train_score25 = model.evaluate(train_X, train_y, verbose=0)
print(train_score25)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score25, math.sqrt(train_score25)))
test_score25 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score25, math.sqrt(test_score25)))

0.0226284585123022
Pontuação de Treino: 0.02 MSE (0.15 RMSE)
Pontuação de teste: 0.02 MSE (0.13 RMSE)


In [13]:
model = Sequential() 

model.add(LSTM(units = 50, return_sequences = True, input_shape = (train_X.shape[1], 1))) 
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = False)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1)) 
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

#es = EarlyStopping(monitor='val_loss',mode='auto',verbose = 1)

model.fit(train_X, train_y, epochs = 100, batch_size = 32, validation_data=(test_X,test_y))

Train on 130 samples, validate on 32 samples
Epoch 1/100
130/130 [==============================] - 22s 169ms/sample - loss: 0.0478 - val_loss: 0.0292
Epoch 2/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0419 - val_loss: 0.0247
Epoch 3/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0363 - val_loss: 0.0206
Epoch 4/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0310 - val_loss: 0.0174
Epoch 5/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0276 - val_loss: 0.0157
Epoch 6/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0243 - val_loss: 0.0154
Epoch 7/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0235 - val_loss: 0.0161
Epoch 8/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0229 - val_loss: 0.0166
Epoch 9/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0229 - val_loss: 0.0169
Epoch 10/100
130/130 [===

130/130 [==============================] - 0s 2ms/sample - loss: 0.0232 - val_loss: 0.0159
Epoch 80/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0229 - val_loss: 0.0160
Epoch 81/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0238 - val_loss: 0.0161
Epoch 82/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0222 - val_loss: 0.0162
Epoch 83/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0232 - val_loss: 0.0162
Epoch 84/100
130/130 [==============================] - 0s 2ms/sample - loss: 0.0234 - val_loss: 0.0162
Epoch 85/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0236 - val_loss: 0.0162
Epoch 86/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0226 - val_loss: 0.0161
Epoch 87/100
130/130 [==============================] - 0s 1ms/sample - loss: 0.0226 - val_loss: 0.0160
Epoch 88/100
130/130 [==============================] - 0s 1ms/sample - loss:

In [14]:
train_score50 = model.evaluate(train_X, train_y, verbose=0)
print(train_score50)
print('Pontuação de Treino: {:.2f} MSE ({:.2f} RMSE)'.format(train_score50, math.sqrt(train_score50)))
test_score50 = model.evaluate(test_X, test_y, verbose=0)
print('Pontuação de teste: {:.2f} MSE ({:.2f} RMSE)'.format(test_score50, math.sqrt(test_score50)))

0.022555690388589237
Pontuação de Treino: 0.02 MSE (0.15 RMSE)
Pontuação de teste: 0.02 MSE (0.13 RMSE)
